## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.21,-159.78,77.00,65,5,9.17,clear sky
1,1,Nikolskoye,RU,59.70,30.79,60.80,82,75,6.71,light intensity shower rain
2,2,Butaritari,KI,3.07,172.79,83.19,63,99,6.46,overcast clouds
3,3,Coihaique,CL,-45.58,-72.07,48.20,61,75,6.93,broken clouds
4,4,Barrow,US,71.29,-156.79,42.80,87,90,24.16,overcast clouds


In [3]:
city_data_df.count()

City_ID                680
City                   680
Country                677
Lat                    680
Lng                    680
Max Temp               680
Humidity               680
Cloudiness             680
Wind Speed             680
Current Description    680
dtype: int64

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.21,-159.78,77.00,65,5,9.17,clear sky
2,2,Butaritari,KI,3.07,172.79,83.19,63,99,6.46,overcast clouds
8,8,Awjilah,LY,29.11,21.29,75.81,48,0,9.64,clear sky
11,11,Kapaa,US,22.08,-159.32,86.00,66,40,13.87,scattered clouds
13,13,Nouadhibou,MR,20.93,-17.03,73.40,83,72,13.87,broken clouds
14,14,Auki,SB,-8.77,160.70,83.95,76,80,5.82,moderate rain
16,16,Sola,VU,-13.88,167.55,86.00,74,75,4.70,broken clouds
21,21,Albany,US,42.60,-73.97,78.01,63,2,5.99,clear sky
25,25,Mareeba,AU,-17.00,145.43,82.40,28,20,16.11,few clouds
26,26,Eydhafushi,MV,5.10,73.07,83.71,66,0,4.68,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                301
City                   301
Country                301
Lat                    301
Lng                    301
Max Temp               301
Humidity               301
Cloudiness             301
Wind Speed             301
Current Description    301
dtype: int64

In [7]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                301
City                   301
Country                301
Lat                    301
Lng                    301
Max Temp               301
Humidity               301
Cloudiness             301
Wind Speed             301
Current Description    301
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,77.00,clear sky,-21.21,-159.78,
2,Butaritari,KI,83.19,overcast clouds,3.07,172.79,
8,Awjilah,LY,75.81,clear sky,29.11,21.29,
11,Kapaa,US,86.00,scattered clouds,22.08,-159.32,
13,Nouadhibou,MR,73.40,broken clouds,20.93,-17.03,
14,Auki,SB,83.95,moderate rain,-8.77,160.70,
16,Sola,VU,86.00,broken clouds,-13.88,167.55,
21,Albany,US,78.01,clear sky,42.60,-73.97,
25,Mareeba,AU,82.40,few clouds,-17.00,145.43,
26,Eydhafushi,MV,83.71,clear sky,5.10,73.07,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [11]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,77.00,clear sky,-21.21,-159.78,Paradise Inn
2,Butaritari,KI,83.19,overcast clouds,3.07,172.79,Isles Sunset Lodge
8,Awjilah,LY,75.81,clear sky,29.11,21.29,
11,Kapaa,US,86.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
13,Nouadhibou,MR,73.40,broken clouds,20.93,-17.03,El Medina
14,Auki,SB,83.95,moderate rain,-8.77,160.70,Auki Motel
16,Sola,VU,86.00,broken clouds,-13.88,167.55,Leumerus Bungalows
21,Albany,US,78.01,clear sky,42.60,-73.97,
25,Mareeba,AU,82.40,few clouds,-17.00,145.43,Trinity Plains Tourist Park
26,Eydhafushi,MV,83.71,clear sky,5.10,73.07,Soneva Fushi


In [18]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(subset = ["Hotel Name"], inplace=True)


hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [24]:
clean_hotel_df = hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng", "Hotel Name"]].copy()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,77.00,clear sky,-21.21,-159.78,Paradise Inn
2,Butaritari,KI,83.19,overcast clouds,3.07,172.79,Isles Sunset Lodge
11,Kapaa,US,86.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
13,Nouadhibou,MR,73.40,broken clouds,20.93,-17.03,El Medina
14,Auki,SB,83.95,moderate rain,-8.77,160.70,Auki Motel


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
#<dt>Description</dt><dd>{Description}</dd>
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
Current_Description = clean_hotel_df["Current Description"]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))